# Diffusion "Self-guidance"

https://dave.ml/selfguidance/

**Use-case:** Method to move/resize objects, replace objects with new ones, change scene backgrounds of diffusion generated images

**Overview:**

- uses the attention maps and activations to steer the image generation
  - both are from the attention layers of the diffusion model<sup>1</sup>
  - these allow us to figure out objects, object position/size...
- using the above we figure out a value to add to the "noise" generated at each step of de-noising<sup>2</sup>
  - since the noise is subtracted from the noisy image to produce a clearer image<sup>3</sup>
  - this extra value effectively changes what the image produced will look like
- math on the "self-guidance" <sup>4</sup>
  - Object position: Computed as center of mass of relevant attention channels
  - Object size: Spatial sum of thresholded attention channel
  - Object shape: Thresholded attention map itself
  - Object appearance: Combination of thresholded attention and spatial activation maps

**Q's:**
- what's an attention map? (medium- priority)
  - perhaps do notes on the recurrent models chapter up to transformers to solidify the attention concept
  - then get this attention map info if still needed
- everything on the third bullet above object position..., no idea what any of those things mean. (medium priority)

<sup><sub>Further contexts:</sub></sup>
- <sup><sub>1. the original diffusion paper (Ho et al.) does not use attention layers, but the ones today (2024) do. ex. Stable Diffusion</sub></sup>
- <sup><sub>2. if i forget this just skim the diffusion folder notes</sub></sup>
- <sup><sub>3. not really "clearer" as in there's a final form for the noisy image thats the same all the time. i just mean getting closer to the formed image by removing a bit more noise.<sub></sup>
- <sup><sub>4. i honestly have no idea what these things mean like "center of mass" of attention channels. i'm considering coming back to this after a firmer grasp on attention, then attention maps. And then see if i get closer to grokking this, since i need a better grasp on attention/recurrent nn fundamentals to advance on involving in ai research regardless of this paper <sub></sup>


## Math of guiding image generation with "self-guidance"

Typical diffusion:

$$
\hat{\epsilon}_t = (1 + s)\epsilon_{\theta}(z_t; t, y) - s\epsilon_{\theta}(z_t; t, \emptyset)
$$

- $\hat{\epsilon}_t$ noise to be removed at step t
- $z_t$ is the current noisy image
- y is the conditioning (like a text prompt)
- $\emptyset$ is no conditioning (gen diffusion as if no text prompt is there<sup>1</sup>)
- $\epsilon_{\theta}$ is the function to predict the noise
  - $\epsilon_{\theta}(z_t; t, y)$ predicts the noise given the conditioning info (from text prompt)
  - $\epsilon_{\theta}(z_t; t, \emptyset)$ predicts the noise given no conditioning

With self-guidance:

$$
\hat{\epsilon}_t = (1 + s)\epsilon_{\theta}(z_t; t, y) - s\epsilon_{\theta}(z_t; t, \emptyset) + v\sigma_t \nabla_{z_t} g(z_t; t, y)
$$

- $v\sigma_t \nabla_{z_t} g(z_t; t, y)$ is the only thing thats different (added)
  - calculating this value for each step allows for "self-guidance" (changing object size, position, etc)

**Q's:**
- why is the first question noise w/ text conditioning minus noise w/o text conditioning? (low+ priority)
  - how subtracting nosie w/o text conditioning help with finding the correct noise amount to remove at that step?
- in the first equation, why is the scalars (1+s) applied? (low- priority)
- in the first equation, what exactly does the math function/equation for $epsilon_{\theta}$ actually look like? (low+ priority)

<sup><sub>Further contexts:</sub></sup>
- <sup><sub>1. though doesn't quite mean "empty" string. perhaps digging into this will unveil what it is mathematically (low priority)</sub></sup>

### Digging into the $v\sigma_t \nabla_{z_t} g(z_t; t, y)$ term

$$v\sigma_t \nabla_{z_t} g(z_t; t, y)$$